# 4. Woordfrequenties - hoe vaak komt een woord voor in een document of een corpus?

Welke woorden komen het meeste voor in een bepaald egodocument? De onderstaande code berekent de frequenties van alle woorden in een enkele tekst. De tekst waarin wordt gezocht wordt bepaald door de waarde van de variabele `egodocument`. De code toont vervolgens de 30 meest voorkomende woorden in deze tekst. De variabele `max` het aantal woorden dat wordt getoond. 

Ook hier is het van belang de stopwoordenlijst te gebruiken om alleen zinvolle frequenties in de resultaten terug te krijgen die te maken hebben met de inhoud, en om niet ter zake doende lidwoorden, bijwoorden en voorzetsels er uit te filteren. 

In [ ]:
from kitlvTdm import *
import os
from os.path import join
import re
import pandas as pd

df = pd.DataFrame(columns = ['word','frequentie', 'total_tokens'])

max = 150 # Het maximum aantal woorden dat later wordt geprint en opgeslagen
path = 'corpus' # Het pad naar de map met de corpus
outfile = "corpus_frequency"

def sortedByValue( dict ):
    return sorted( dict , key=lambda x: dict[x], reverse = True)

# de totale frequenties worden opgeslagen in een dictionary
freqTotal = dict() 



# Hier worden het aantal woorden (tokens) en documenten bijgehouden
tokensTotal = 0
egodocuments = 0

for i, file in enumerate(os.listdir(path)):
    
    percentage_done = i / len(os.listdir(path)) * 100
    print("percentage done: {:.2f}%".format(percentage_done), end = "\r")
    
    if re.search( 'txt$' , file ):
        egodocuments += 1
        
        # bereken per woord hoe vaak het voor komt
        freq = calculateWordFrequencies( join( path, file ) )
        
        # Haal de stopwoorden eruit
        freq = removeStopwords(freq)

        for word in freq:
            if word  in freqTotal: # Als het woord al in freqTotal staat: tel extra aantal erbij op
                freqTotal[word] += freq[word]
            else: # anders, plaats het woord in freqTotal
                freqTotal[word] = freq[word]
        
        tokensTotal += numberOfTokens( join( path, file ) )

        if i > 50:
            break
            
for count, word in enumerate(sortedByValue(freqTotal)):
    print( f' { word } => { freqTotal[word] / tokensTotal }' )
    
    df = df.append({'word' : word, 'frequency' : freqTotal[word] / tokensTotal, 'total_tokens': tokensTotal}, ignore_index = True)
    
    if count == max:
        break
        
df.to_excel(f"{outfile}.xlsx")
print(f"Excel file saved as {outfile}.xlsx") 


**Oefening 4: Bepaal de meest frequente woorden in een van de egodocumenten in het corpus van "Soldaat in Indonesië". Experimenteer met verschillende waarden voor de variabelen `egodocument` en `max`.**

De woordfrequenties geven deels een vertekend beeld, omdat in veel memoires letterlijk passages of hele delen van dagboeken uit de tijd zelf worden gebruikt. Eigenlijk zou je die boeken die verhoudingsgewijze veel van deze teksten bevatten moeten kunnen isoleren, zodat de boeken zonder deze passages kunnen worden geanalyseerd. Dat kan alleen door extra codes aan de publicaties toe te voegen die dit verschil aangeeft. Die laag is er nu nog niet, maar staat wel in de planning.

Veranderingen in woordgebruik zijn gebonden aan tijd en veranderingen in de samenleving. Omdat dit corpus zich uitstrekt vanaf de periode van het conflict zelf (1945-1949) tot aan 2017, is het interessant om het voorkomen van bepaalde termen chronologisch te vergelijken. De onderstaande code verdeelt het corpus in perioden van 5 jaar, en berekent vervolgens de woordfrequenties voor de egodocumenten die in deze verschillende tijdvakken verschenen. Hierbij moet wel de kanttekening worden geplaatst dat niet alle egodocumenten konden worden gedateerd. Bij deze analyse worden de teksten die nog niet zijn gedateerd genegeerd. Verder is het uiteraard ook zo dat er een onevenredige verdeling is van het aantal boeken over deze perioden. De absolute frequenties kunnen daardoor niet zonder meer worden vergeleken. OM de frequenties toch vergelijkbaar te maken zijn de absolute tellingen steeds gedeeld door het totaal aantal woorden in de egoducmenten uit de verschillende perioden.

De lengte van de geanlyseerde periode kan overgens worden aangepast via de variabele `period_length`. 

De resultaten worden getoond in dit Notebook, maar worden eveneens weggeschreven in een bestand met de naam `frequency_chronological.csv`. 

In [ ]:
from kitlvTdm import *
import os
from os.path import join
import re
import pandas as pd

# specificeer hier de start en eind datum om te doorzoeken
start = 1930
end = 2014

# specificeer hier de lengte van de perioden en het aantal woorden voor printen/opslaan
period_length = 5
max = 150

path = 'corpus'
outfile = f"woordfrequenties_perioden_{start}_{end}_{period_length}"

df = pd.DataFrame(columns = ["period", "word", "frequency", "total_tokens_period"])

def sortedByValue( dict ):
    return sorted( dict , key=lambda x: dict[x])

# Hier worden de verschillende tijdsperioden berekent
intervals = []
for year in range( start , end , period_length ):
    intervals.append(year)
    

for year in intervals:
    year_from = year
    year_to =  period_length - 1 + year
    freqTotal = dict()
    
    tokensTotal = 0
    egodocuments = 0

    for file in os.listdir(path):
        if re.search( 'txt$' , file ):
            year = showYear( file )
            if year != "" and len(year) == 4:
                year = int(year)
                if year >= year_from and year <= year_to:
                    egodocuments += 1
                    freq = calculateWordFrequencies( join( path, file ) )
                    
                    for word in freq:
                        if word  in freqTotal: # Als het woord al in freqTotal staat: tel extra aantal erbij op
                            freqTotal[word] += freq[word]
                        else: # anders, plaats het woord in freqTotal
                            freqTotal[word] = freq[word]
                    
                    tokensTotal += numberOfTokens( join( path, file ) )

    print( f'\n{ year_from }-{ year_to }\n{tokensTotal} words in total in {egodocuments} egodocuments\n\n' )


    freqTotal = removeStopwords( freqTotal )

    count = 0
    for word in reversed( sortedByValue(freqTotal) ):
        
        df = df.append({"period" : f"{year_from}-{year_to}", "word" : word,
                        "frequency" : freqTotal[word] / tokensTotal, 
                        "total_tokens_period": tokensTotal }, ignore_index = True)
        
        print( f' { word } => { freqTotal[word] / tokensTotal }' )
        count += 1
        if count == max:
            break

df.to_excel(f"{outfile}.xlsx")
print(f"Excel file saved as {outfile}.xlsx")

**Oefening 4: Probeer met behulp van de bovenstaande code te verkennen hoe het woordgebruik zich ontwikkelde over de loop van de afgelopen decennia. Verander hiervoor de waarde van de variabelen `period_length` en `nr_words`.**

Woordsoorten als zelfstandige naamwoorden, bijvoeglijke naamwoorden en werkwoorden drukken zijn over het algemeen het meest bepalend voor de betekenis van zinnen. Het kan daarom nuttig en informatief zijn om frequentie-analyses te beperken tot dit soort woorden. In de onderstaande cellen worden uitsluitend de zelfstandige naamwoorden, bijvoeglijke naamwoorden en werkwoorden geteld, door gebruik te maken van de hierboven al genoemde module `nltk`. Deze module richt zich normaal gesproken op Engelstalige teksten. Om `nltk` ook toe te kunnen passen op Nederlandstalige teksten moet eerst de onderstaande code worden uitgevoerd. 

In [ ]:
import nltk
nltk.download('alpino')

from nltk.corpus import alpino as alp
from nltk.tag import UnigramTagger, BigramTagger
training_corpus = alp.tagged_sents()
unitagger = UnigramTagger(training_corpus)
bitagger = BigramTagger(training_corpus, backoff=unitagger)
pos_tag = bitagger.tag

Als de installatie van de Nederlandstalige variant van `nltk` geen problemen opleverde, kan de onderstaande, meer gerichte frequentie-analyse worden uitgevoerd. Let er hierbij op dat het toekennen van grammaticale categorieën wel enige rekenkracht vergt. Het uitvoeren van de code kan dus enige tijd in beslag nemen.  

In [ ]:
from kitlvTdm import *
import os
from os.path import join
from nltk.tokenize import sent_tokenize, word_tokenize

nr_words = 100
tokens_total = 0
path = 'corpus'
egodocument = '03391.txt'

def sortedByValue( dict ):
    return sorted( dict , key=lambda x: dict[x])



out = open( 'frequency_POS.csv' , 'w' , encoding = 'utf-8' )
out.write( 'word,frequency\n' )



freqTotal = dict()

countFile = 0 

for file in os.listdir(path):
    if re.search( 'txt$' , file ):
        countFile += 1
        if file == egodocument:
        #print( '\rReading {} ... ({}/577)'.format(file , countFile ) )
            with open( join( path, file ), encoding = "utf-8") as fileName:
                print(fileName)
                fullText = fileName.read()
                sent = sent_tokenize(fullText)
                for s in sent:
                    words = word_tokenize(s)
                    pos = pos_tag(words)
                    tokens_total += len(words)
                    for p in pos:
                        if p[1] is not None:
                            if re.search( r'^(adj)|(noun)|(verb)' , p[1] ):
                                freqTotal[ p[0]  ] = freqTotal.get( p[0] ,0 ) + 1


count = 0                                
freqTotal = removeStopwords( freqTotal )
for word in reversed( sortedByValue(freqTotal) ):
    out.write( f'{word},{freqTotal[word]}\n' )
    print( f' { word } => { freqTotal[word] / tokens_total }' )
    count += 1
    if count == nr_words:
        break

out.close()

**Oefening 5: Probeer een lijst te generen van de 150 meest frequente zelfstandige naamwoorden, bijvoeglijke naamwoorden en werkwoorden.**